In [40]:
import pandas as pd
import config
import numpy as np
from requests import post
import json
from utils import t_student_test
from config import AB_DIVIDE_MONTH, FEATURES, TARGET

In [31]:
data = pd.read_csv('../data/final_data_filtered.csv', index_col='Unnamed: 0')
data = data.dropna()

In [32]:
data_ab = data.loc[data['month'] >= AB_DIVIDE_MONTH]

In [33]:
API_URL = 'http://127.0.0.1:8000/'
post(API_URL + 'reset/')

<Response [200]>

In [34]:
X = data_ab[FEATURES + ['user_id']]
y = data_ab[TARGET]

y = y.rename(columns=({'premium_purchased': 'true_premium_purchased',
                       'premium_purchased_this_month': 'true_premium_purchased_this_month'}))

In [35]:
for index, row in X.iterrows():
    response = post(API_URL + 'ab/', row.to_json())

# Wykonano eksperymenty, łączymy dane z serwera z prawdziwymi danymi.

In [37]:
ab_results = pd.read_csv('../microservice/ab_results.csv')
nn_results = ab_results.loc[ab_results['model'] == 'nn']
lr_results = ab_results.loc[ab_results['model'] == 'lr']

In [45]:
group_a = nn_results['premium_purchased']
group_b = lr_results['premium_purchased']

t_student_test(group_a, group_b)

T-statistic: 138.98581719813018
P-value: 0.0
Odrzucamy hipotezę zerową. Istnieje istotna różnica między grupami.
Grupa A jest lepsza niż grupa B.


In [46]:
group_a = nn_results['premium_purchased_this_month']
group_b = lr_results['premium_purchased_this_month']

t_student_test(group_a, group_b)

T-statistic: -138.92225086705102
P-value: 0.0
Odrzucamy hipotezę zerową. Istnieje istotna różnica między grupami.
Grupa B jest lepsza niż grupa A.
